In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as mpl

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
strategy_name = 'A2C_V111_12041115_25M'
portfolio_prefix = f'../results/cs/A2C/{strategy_name}'
actions_suffix = '_actions.csv'
value_suffix = '_portfolio_value.csv'
stock_prices = '../datasets/thesis/1d/all_1d.csv'
benchmark_returns_1d = '../datasets/thesis/benchmark/1d_benchmark_returns.csv'
benchmark_returns_1h = '../datasets/thesis/benchmark/1h_benchmark_returns.csv'


In [ ]:
df = pd.read_csv(f"{portfolio_prefix}{value_suffix}", index_col=0)
df.index = pd.to_datetime(df['date'])
df = df.drop('date', axis=1)
df['return'] = df['account_value'].pct_change() + 1
df['equity'] = (df['return']).cumprod()
df.loc[df.iloc[0].name]['return'] = 1
df.loc[df.iloc[0].name]['equity'] = 1

In [ ]:
df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
df2.index = pd.to_datetime(df2.index)

In [ ]:
df3 = pd.read_csv(stock_prices)
df3['date'] = pd.to_datetime(df3['date'])
# get or column with close price for each stock per tick
df3 = df3.pivot(index="date", columns="tic", values="close")
# limit dataset to portfolio range
df3 = df3[(df3.index >= df2.index[0]) & (df3.index <= df2.index[-1])]
# get relative change from price
df3 = df3.pct_change()
# set first row to 0
df3.loc[df3.iloc[0].name] = 0
# get cumulative return for each stock
df3 = (df3 + 1).cumprod()

In [ ]:
names = df3.columns
# names = ['AAPL', 'AMGN', 'MRK']
# names = ['AAPL']

fig = make_subplots(specs=[[{"secondary_y": True}]])

for name in names:
    fig.add_trace(go.Scatter(x=df3.index,y=df3[name],name=name),secondary_y=False)
    sell_idx = df2[df2[name] < 0][name].index
    buy_idx = df2[df2[name] > 0][name].index
    fig.add_trace(go.Scatter(x=buy_idx,y=df3.loc[buy_idx][name],name='buy', mode = 'markers', marker_color='rgba(50, 205, 50, .9)',marker_size=5, showlegend=False),secondary_y=False)
    fig.add_trace(go.Scatter(x=sell_idx,y=df3.loc[sell_idx][name],name=f'sell', mode = 'markers', marker_color='rgba(255, 0, 0, .9)',marker_size=5, showlegend=False),secondary_y=False)

fig.update_layout(title_text='Buy / Sell Actions', height=800)
fig.show()
fig.write_html(f"{portfolio_prefix}_actions_overview.html")

In [ ]:
df_benchmark = pd.read_csv(f"{benchmark_returns_1h}")
df_benchmark.index = pd.to_datetime(df_benchmark['date'])
df_benchmark = df_benchmark.drop('date', axis=1)
df_benchmark = df_benchmark[(df_benchmark.index >= df_strategy.index[0]) & (df_benchmark.index <= df_strategy.index[-1])]
# df_benchmark = df_benchmark.resample('1h').asfreq().fillna(1)

In [ ]:
df['strategy'] = df['return']
df_strategy = df[['strategy']] 
df_benchmark = df_benchmark[(df_benchmark.index >= df_strategy.index[0]) & (df_benchmark.index <= df_strategy.index[-1])]
df_returns = df_strategy.merge(df_benchmark, how='left', left_index=True, right_index=True)
df_returns = df_returns.cumprod()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

for name in df_returns.columns:
    fig.add_trace(go.Scatter(x=df_returns.index,y=df_returns[name],name=name),secondary_y=False)
fig.update_layout(title_text='Strategy Performance compared to benchmarks', height=600)
fig.show()
fig.write_html(f"{portfolio_prefix}_performance_compare.html")

In [ ]:
df_strategy